In [1]:
import pandas as pd
import os
import time
from tqdm import tqdm

# --- CẤU HÌNH ---
# Folder để lưu file sau khi xử lý (sequence dài 510)
PREPARED_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data"

# Input hiện tại là 601, Output mong muốn cho SpliceBERT là 510
TARGET_LEN = 510 

# --- HÀM XỬ LÝ LOGIC (GIỮ NGUYÊN) ---
def crop_sequence_worker(sequence):
    """Hàm cắt chuỗi từ 601bp xuống 510bp, giữ tâm ở giữa."""
    if pd.isna(sequence):
        return "N" * TARGET_LEN
    
    seq_len = len(sequence)
    if seq_len < TARGET_LEN:
        return sequence + ("N" * (TARGET_LEN - seq_len))
    
    start_idx = (seq_len - TARGET_LEN) // 2
    end_idx = start_idx + TARGET_LEN
    return sequence[start_idx:end_idx].upper()

def diagnose_splice_sites(df, sample_size=5):
    """Kiểm tra xem sau khi cắt, cặp GT/AG có còn nằm ở giữa (index 255) không."""
    print(f"\n{'Type':<10} | {'Window center (idx 253-259)':<30} | {'Found?'}")
    print("-" * 70)
    center_idx = TARGET_LEN // 2 
    
    for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
        samples = df[df['Splicing_types'] == label]
        if len(samples) == 0: continue
        
        test_batch = samples.sample(min(sample_size, len(samples)))
        for _, row in test_batch.iterrows():
            seq = row['sequence']
            window = seq[center_idx - 5 : center_idx + 5] 
            motif = seq[center_idx : center_idx + 2]
            target = "GT" if label == 1 else "AG"
            is_match = "✅" if motif == target else "❌"
            display_win = f"...{seq[center_idx-2:center_idx]} [{motif}] {seq[center_idx+2:center_idx+4]}..."
            print(f"{name:<10} | {display_win:<30} | {is_match} (Got: {motif})")

def process_file_list(file_paths):
    """Hàm chạy vòng lặp qua danh sách file do người dùng nhập"""
    os.makedirs(PREPARED_FOLDER, exist_ok=True)
    
    print(f"📋 Tìm thấy {len(file_paths)} file cần xử lý.\n")

    for full_path in file_paths:
        # 1. Kiểm tra file tồn tại
        if not os.path.exists(full_path):
            print(f"❌ LỖI: Không tìm thấy file tại đường dẫn:\n   {full_path}\n   -> Bỏ qua file này.\n")
            continue
            
        file_name = os.path.basename(full_path) # Lấy tên file (vd: test_1_1_1.csv)
        output_path = os.path.join(PREPARED_FOLDER, file_name)
        
        print(f"🚀 Đang xử lý: {file_name} ...")
        start_time = time.time()
        
        # 2. Đọc file
        try:
            df = pd.read_csv(full_path)
        except Exception as e:
            print(f"❌ Lỗi đọc CSV: {e}")
            continue

        # 3. Cắt chuỗi
        tqdm.pandas(desc="   Cropping")
        df['sequence'] = df['sequence'].progress_apply(crop_sequence_worker)
        
        # 4. Kiểm tra độ dài
        df['len_check'] = df['sequence'].apply(len)
        if (df['len_check'] != TARGET_LEN).any():
            print("   ⚠️ Warning: Có dòng không đủ độ dài 510!")
        df = df.drop(columns=['len_check'])

        # 5. Diagnose & Lưu
        diagnose_splice_sites(df)
        df.to_csv(output_path, index=False)
        
        duration = time.time() - start_time
        print(f"✅ Đã lưu vào: {output_path}")
        print(f"   Tốc độ: {len(df)/duration:.2f} seq/s\n")
        print("-" * 50)

# --- PHẦN NHẬP DỮ LIỆU CỦA BẠN ---
if __name__ == "__main__":
    # 👇 BẠN HÃY PASTE ĐƯỜNG DẪN CÁC FILE CỦA BẠN VÀO DANH SÁCH NÀY
    # Lưu ý: Dùng chữ r trước dấu nháy (r"...") để Windows không bị lỗi đường dẫn
    
    INPUT_FILES = [
        r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_1_1_1.csv",
        r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_2_1_1.csv",
        r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_4_1_1.csv",
        r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_10_1_1.csv",
        r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_data.csv"
    ]

    print("🏁 BẮT ĐẦU QUÁ TRÌNH XỬ LÝ DỮ LIỆU...")
    process_file_list(INPUT_FILES)
    print("🎉 HOÀN TẤT!")

🏁 BẮT ĐẦU QUÁ TRÌNH XỬ LÝ DỮ LIỆU...
📋 Tìm thấy 5 file cần xử lý.

🚀 Đang xử lý: test_1_1_1.csv ...


   Cropping: 100%|██████████| 26310/26310 [00:00<00:00, 1095437.06it/s]


Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...AG [GT] AA...               | ✅ (Got: GT)
Donor      | ...TG [GT] GA...               | ✅ (Got: GT)
Donor      | ...GT [GT] GT...               | ✅ (Got: GT)
Donor      | ...AG [GT] AA...               | ✅ (Got: GT)
Donor      | ...AG [GT] GA...               | ✅ (Got: GT)
Acceptor   | ...CA [GA] TA...               | ❌ (Got: GA)
Acceptor   | ...CA [GG] AG...               | ❌ (Got: GG)
Acceptor   | ...TA [GG] TT...               | ❌ (Got: GG)
Acceptor   | ...CA [GG] AT...               | ❌ (Got: GG)
Acceptor   | ...CA [GA] TC...               | ❌ (Got: GA)


✅ Đã lưu vào: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_1_1_1.csv
   Tốc độ: 124232.65 seq/s

--------------------------------------------------
🚀 Đang xử lý: test_2_1_1.csv ...


   Cropping: 100%|██████████| 35132/35132 [00:00<00:00, 1080096.23it/s]

...


Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...AG [GT] AG...               | ✅ (Got: GT)
Donor      | ...AG [GT] AG...               | ✅ (Got: GT)
Donor      | ...AG [GT] GA...               | ✅ (Got: GT)
Donor      | ...AG [GT] AC...               | ✅ (Got: GT)
Donor      | ...TG [GT] GA...               | ✅ (Got: GT)
Acceptor   | ...CA [GG] GA...               | ❌ (Got: GG)
Acceptor   | ...TA [GC] TT...               | ❌ (Got: GC)
Acceptor   | ...CA [GA] AG...               | ❌ (Got: GA)
Acceptor   | ...TA [GG] TT...               | ❌ (Got: GG)
Acceptor   | ...TA [GG] AT...               | ❌ (Got: GG)
✅ Đã lưu vào: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_4_1_1.csv
   Tốc độ: 131727.82 seq/s

--------------------------------------------------
🚀 Đang xử lý: test_10_1_1.csv ...


   Cropping: 100%|██████████| 105708/105708 [00:00<00:00, 1061380.38it/s]



Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...AG [GT] GG...               | ✅ (Got: GT)
Donor      | ...TG [GT] GA...               | ✅ (Got: GT)
Donor      | ...CT [GT] AA...               | ✅ (Got: GT)
Donor      | ...CT [GT] TT...               | ✅ (Got: GT)
Donor      | ...AG [GT] GG...               | ✅ (Got: GT)
Acceptor   | ...AA [GC] CC...               | ❌ (Got: GC)
Acceptor   | ...GA [GG] AC...               | ❌ (Got: GG)
Acceptor   | ...CA [GG] GA...               | ❌ (Got: GG)
Acceptor   | ...CA [GC] CT...               | ❌ (Got: GC)
Acceptor   | ...CA [GG] TA...               | ❌ (Got: GG)
✅ Đã lưu vào: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_10_1_1.csv
   Tốc độ: 130031.11 seq/s

--------------------------------------------------
🚀 Đang xử lý: test_data.csv ...


   Cropping: 100%|██████████| 938297/938297 [00:00<00:00, 1094915.47it/s]



Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...AG [GT] GA...               | ✅ (Got: GT)
Donor      | ...CG [GT] AA...               | ✅ (Got: GT)
Donor      | ...AG [GT] AA...               | ✅ (Got: GT)
Donor      | ...CG [GT] GA...               | ✅ (Got: GT)
Donor      | ...AA [GT] GA...               | ✅ (Got: GT)
Acceptor   | ...AA [GA] AG...               | ❌ (Got: GA)
Acceptor   | ...CA [GC] AC...               | ❌ (Got: GC)
Acceptor   | ...CA [GG] TT...               | ❌ (Got: GG)
Acceptor   | ...GA [GG] GG...               | ❌ (Got: GG)
Acceptor   | ...TA [GA] AA...               | ❌ (Got: GA)
✅ Đã lưu vào: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_data.csv
   Tốc độ: 139135.48 seq/s

--------------------------------------------------
🎉 HOÀN TẤT!
